In [ ]:
#I am using this notebook as practice to follow along with evolving hockeys tutorial
#on how they calculate their expected goal value. link to the tutorial is here
#https://evolving-hockey.com/blog/a-new-expected-goals-model-for-predicting-goals-in-the-nhl/

In [1]:
import pandas as pd
import numpy as np

In [2]:
#df_import = pd.read_csv('data/washington_sample.csv')
df_import = pd.read_csv('data/march_4th_sample.csv')

In [ ]:
filter_list = ['FAC', 'HIT', 'MISS', 'SHOT', 'BLOCK', 'GIVE', 'TAKE', 'GOAL']
df_event_fill = df_import[df_import['event_type'].isin(filter_list)]

In [ ]:
df_event_fill.head()

In [3]:
#df = df_event_fill.copy()
df = df_import.copy()

In [4]:
#This page was helpful https://stackoverflow.com/questions/39109045/numpy-where-with-multiple-conditions
#Conditions for extracting the zone
col = 'event_description'
conditions = [df[col].str.contains('Off. Zone', na=False), df[col].str.contains('Def. Zone', na=False), 
              df[col].str.contains('Neu. Zone', na=False)]
choices = ['Off', 'Def','Neu']
fenwick = ['SHOT', 'MISS', 'GOAL']
corsi = ['SHOT', 'MISS', 'GOAL', 'BLOCK']
correction = ['Tip-In', 'Wrap-around', 'Deflected']

In [5]:
df['event_distance'] = ((89 - abs(df['coords_x']))**2 + df['coords_y']**2)**(1/2)
df['event_angle'] = abs(np.arctan(df['coords_y'] / (89 - abs(df['coords_x']))) * (180 / np.pi))
df['pre_event_zone'] = np.select(conditions, choices, default=np.nan)
df['pre_event_zone_2'] = np.where((df.pre_event_zone == 'Def') & 
                                  (df.event_type == 'BLOCK'), 
                                  'Off',df.pre_event_zone)
df['pre_event_zone_3'] = np.where((df.event_type.isin(fenwick)) & 
                                  (df.event_zone == 'Def') & 
                                  (df.pbp_distance <= 64),
                                  'Off',df.pre_event_zone)
df['event_distance_long'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.pbp_distance > 89) &
                                     (df.coords_x < 0) &
                                     (~df.event_detail.isin(correction)) &~
                                     ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                     (((abs(df.coords_x)+89)**2) + df.coords_y**2)**(1/2)
                                     ,df.event_distance)
df['event_distance_long_2'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.pbp_distance > 89) &
                                     (df.coords_x > 0) &
                                     (~df.event_detail.isin(correction)) &~
                                     ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                     (((df.coords_x+89)**2) + df.coords_y**2)**(1/2)
                                     ,df.event_distance)
df['event_angle_long'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.pbp_distance > 89) &
                                     (df.coords_x < 0) &
                                     (~df.event_detail.isin(correction)) &~
                                     ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                     abs(np.arctan(df['coords_y']/(abs(df['coords_x'])+89))*(180 / np.pi))
                                     ,df.event_angle)
df['event_angle_long_2'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.pbp_distance > 89) &
                                     (df.coords_x > 0) &
                                     (~df.event_detail.isin(correction)) &~
                                     ((df.pbp_distance > 89) & (df.event_zone == 'Off')),
                                     abs(np.arctan(df['coords_y']/(df['coords_x']+89))*(180 / np.pi))
                                     ,df.event_angle)
df['event_zone_long'] = np.where((df.event_type.isin(fenwick)) & 
                                     (df.event_zone == 'Def') &
                                     (df.pbp_distance <= 64),
                                     'Off',
                                     df.event_zone)
df['game_strength_state'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.home_team),
                                     'Ev1',
                                     df.game_strength_state)
df['game_strength_state'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.away_team),
                                     '1vE',
                                     df.game_strength_state)
df['home_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.home_team),
                                     1,
                                     df.home_skaters)
df['home_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.away_team),
                                     0,
                                     df.home_skaters)
df['away_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.home_team),
                                     0,
                                     df.away_skaters)
df['away_skaters'] = np.where((df.event_description.str.contains('penalty shot', na=False)) &
                                     (df.event_team == df.away_team),
                                     1,
                                     df.away_skaters)

In [6]:
event_interest = ["FAC", "GOAL", "BLOCK", "SHOT", "MISS", "HIT", "TAKE", "GIVE"]
filter_1 = df.event_type.isin(event_interest)
filter_2 = df.game_period < 5
filter_3 = ~df.event_description.str.contains('penalty shot', na=False)
filter_4 = df.coords_x.isna()
filter_5 = df.coords_y.isna()
filter_6 = ~((df.coords_x == 0) &
             (df.coords_y == 0) &
             (df.event_type.isin(corsi)) &
             ((df.pbp_distance != 90) & (df.event_type.isin(fenwick))))

In [7]:
#df_2 = df[filter_1 & filter_2 & filter_3 & filter_4 & filter_5 & filter_6].copy()
df = df[filter_1 & filter_2 & filter_3 & filter_6].copy()

In [8]:
df.dropna(subset=['coords_x','coords_y'],inplace=True)

In [14]:
df = df.groupby(['season', 'game_id', 'game_period']).apply(lambda x: x.sort_values(['event_index']))

In [18]:
df['seconds_since_last'] = df.game_seconds - df.game_seconds.shift(1)
df['event_type_last'] = df.event_type.shift(1)
df['event_team_last'] = df.event_team.shift(1)
df['event_strength_last'] = df.event_strength.shift(1)
df['coords_x_last'] = df.coords_x.shift(1)
df['coords_y_last'] = df.coords_y.shift(1)

In [16]:
df.seconds_since_last.head()

season    game_id     game_period    
20202021  2020020163  1            6      NaN
                                   7     43.0
                                   11    24.0
                                   12    11.0
                                   15     0.0
Name: seconds_since_last, dtype: float64

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.game_period.head()